In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
spotify= pd.read_csv('data/spotify_limpio_train.csv')

In [3]:
spotify.dtypes

Artist             object
Track              object
Album              object
Album_type         object
Stream            float64
Danceability      float64
Energy            float64
Key                 int64
Key_name           object
Loudness          float64
Speechiness       float64
Acousticness      float64
Liveness          float64
Valence           float64
Tempo             float64
Tempo_category     object
Duration_ms       float64
Licensed             bool
official_video       bool
dtype: object

In [4]:
spotify.head()

,Artist,Track,Album,Album_type,Stream,Danceability,Energy,Key,Key_name,Loudness,Speechiness,Acousticness,Liveness,Valence,Tempo,Tempo_category,Duration_ms,Licensed,official_video
0,KAYTRANADA,LITE SPOTS,99.9%,album,29240851.0,0.884,0.549,1,C# / Db,-11.683,0.4710,0.03460,0.112,0.394,120.461,Moderato / Allegretto,230920.0,True,True
1,Rita Ora,Follow Me,Follow Me,single,82813284.0,0.673,0.729,6,F# / Gb,-4.879,0.0496,0.08300,0.100,0.675,122.023,Allegro,169672.0,True,True
2,Big Sean,Bounce Back,I Decided.,album,666145000.0,0.780,0.575,1,C# / Db,-5.628,0.1390,0.10600,0.129,0.273,81.502,Andante,222360.0,True,True
3,Dave Matthews Band,#41,Crash,album,34528391.0,0.577,0.726,4,E,-8.011,0.0299,0.00199,0.158,0.764,107.416,Andante,399800.0,True,True
4,Ray Dalton,Call It Love - Klingande Remix,Call It Love (Klingande Remix),single,1153344.0,0.671,0.687,10,A# / Bb,-8.669,0.0502,0.01110,0.504,0.623,121.992,Allegro,149508.0,True,True


El dataset original contiene varias columnas de tipo texto (por ejemplo, nombre de la canción, artista, enlaces o descripciones) o booleanas, que no pueden ser utilizadas directamente al usar regresión lineal y como son variables que identifican de manera única las canciones las vamos a eliminar.


In [5]:
spotify_sin_string= spotify.drop(['Artist', 'Track', 'Album', 'Album_type', 'Licensed', 'official_video', 'Tempo_category', 'Key_name'], axis=1)

## Modelo Baseline

Entrenamos regresión lineal utilizando las variables
disponibles sin aplicar transformaciones ni escalado. Los coeficientes obtenidos
presentan valores elevados, lo cual es esperable debido a la gran escala de la variable
objetivo (Stream) y de algunas variables explicativas.

In [6]:
X = spotify_sin_string.drop('Stream', axis=1)
y = spotify_sin_string['Stream']

model = LinearRegression()
model.fit(X, y)

y_pred_sklearn = model.predict(X)

In [7]:
print("\nParámetros (sklearn):")
print(f"w1 = {model.coef_[0]:.3f}, w2 = {model.coef_[1]:.3f}, b = {model.intercept_:.3f}")


Parámetros (sklearn):
w1 = 57608001.677, w2 = -133233504.731, b = 318619514.529


In [8]:
# Mostrar MSE final de ambos modelos
print("MSE (sklearn):", mean_squared_error(y, y_pred_sklearn))

MSE (sklearn): 5.801035521333892e+16


El error cuadrático medio (MSE) obtenido es muy alto, lo que se explica por el hecho de
que el error se eleva al cuadrado y la variable objetivo (stream) se mide en millones. Este resultado no se interpreta como un buen rendimiento, sino como una
referencia inicial que servirá para comparar modelos posteriores tras aplicar validación cruzada.

### Validación cruzada

El modelo baseline se evalúa mediante validación cruzada de 5 folds sobre el conjunto de
entrenamiento. Dado que el problema es de regresión, se utiliza el RMSE como métrica de
evaluación

In [9]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(
    model,
    X,
    y,
    cv=5,
    scoring="neg_root_mean_squared_error"
)

rmse_scores = -scores

print("RMSE por fold:", rmse_scores)
print("RMSE medio:", rmse_scores.mean())

RMSE por fold: [2.47433990e+08 2.25708475e+08 2.53613011e+08 2.37653481e+08
 2.39769780e+08]
RMSE medio: 240835747.2392725


Se obtiene un RMSE medio de
aproximadamente 244 millones de reproducciones. Esto implica que, de media, las
predicciones del modelo se desvían en ese orden de magnitud respecto al valor real.
Dado que la variable objetivo presenta valores muy elevados y una distribución muy
asimétrica, este resultado es esperable.

Usamos este valor como referencia para compararlo y ver si coseguimos reducirlo.

### Feature engineering

Probamos usando log(Stream) porque la popularidad musical tiene una distribución de cola larga,
y el log reduce la influencia de valores extremos y mejora el comportamiento del modelo.

No hay desequilibrio de clases, pero sí una distribución del target muy asimétrica,
típica de problemas de popularidad por lo que se aplican transformaciones como el logaritmo

Dado que el dataset no presenta una dimensión temporal,
no se aplican técnicas de agregación temporal ni lags.

In [10]:
import numpy as np

spotify_sin_string["log_stream"] = np.log1p(spotify_sin_string["Stream"])


In [11]:
X = spotify_sin_string.drop(["Stream", "log_stream"], axis=1)
y = spotify_sin_string["log_stream"]

Función para evaluar un modelo con validación cruzada (RMSE). Hemos añadido un apartado condicional para ver si aplicamos o no la normalización a nuestra función. 

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

def baseline_model(X, y, cv=5, scale= False):

    if scale:
        model = Pipeline([
            ("scaler", StandardScaler()),
            ("model", LinearRegression())
        ])
    else:
        model = LinearRegression()
    
    scores = cross_val_score(
        model,
        X,
        y,
        cv=cv,
        scoring="neg_root_mean_squared_error"
    )
    
    rmse_scores = -scores
    
    print("RMSE por fold:", rmse_scores)
    print("RMSE medio:", rmse_scores.mean())
    
    return rmse_scores.mean()


El modelo sin escalado

In [13]:
baseline_model(X,y,5)

RMSE por fold: [1.59790928 1.59809134 1.5808532  1.57133174 1.58387866]
RMSE medio: 1.5864128452115591


np.float64(1.5864128452115591)

Modelo con escalado

In [14]:
baseline_model(X,y,5, scale=True)

RMSE por fold: [1.59790928 1.59809134 1.5808532  1.57133174 1.58387866]
RMSE medio: 1.5864128452115582


np.float64(1.5864128452115582)

Vemos que no hay mucha diferencia así que vamos a continuar sin el escalado.

La normalización de las variables
explicativas no produce mejoras significativas en el error. Vamos a estudiar si las variables son muy relevantes

Analizamos la relevancia individual de
algunas variables.

In [15]:
X_sin_loudness = spotify_sin_string.drop(["Stream", "log_stream", 'Loudness'], axis=1)
y = spotify_sin_string["log_stream"]

In [16]:
baseline_model(X_sin_loudness,y,5)


RMSE por fold: [1.61780051 1.61383981 1.599659   1.59067777 1.60250881]
RMSE medio: 1.6048971782222956


np.float64(1.6048971782222956)

Al eliminar *Loudness*, el RMSE
medio aumenta respecto al modelo base, lo que indica que esta variable aporta
información relevante al modelo. Por tanto, se decide mantener *Loudness* dentro del
conjunto final de variables.

In [17]:
X_sin_tempo = spotify_sin_string.drop(["Stream", "log_stream", 'Tempo'], axis=1)
y = spotify_sin_string["log_stream"]

In [18]:
baseline_model(X_sin_tempo,y,5)

RMSE por fold: [1.59887947 1.59885969 1.58054647 1.57111144 1.58503899]
RMSE medio: 1.586887210875511


np.float64(1.586887210875511)

Probamos eliminar *Tempo* y vemos que no aporta
ninguna mejora en el rendimiento, la mantenemos

ahora evaluamos el
impacto de eliminar simultáneamente las variables *Liveness* y *Duration_ms*

In [19]:
X_sin_liv_inst = spotify_sin_string.drop(["Stream", "log_stream", 'Liveness', 'Duration_ms'], axis=1)
y = spotify_sin_string["log_stream"]

In [20]:
baseline_model(X_sin_liv_inst,y,5)

RMSE por fold: [1.59950251 1.59890636 1.58215967 1.57220259 1.58440615]
RMSE medio: 1.587435455534369


np.float64(1.587435455534369)

El RMSE medio no mejora respecto al modelo base

In [21]:
X_sin_danc = spotify_sin_string.drop(["Stream", "log_stream", 'Danceability'], axis=1)
y = spotify_sin_string["log_stream"]

In [22]:
baseline_model(X_sin_danc,y,5)

RMSE por fold: [1.59834869 1.59872097 1.58037831 1.5722181  1.58420591]
RMSE medio: 1.5867743958758704


np.float64(1.5867743958758704)

Tampoco mejora al eliminar Danceability

se evalúa la eliminación
simultánea de *Speechiness* y *Acousticness*

In [23]:
X_sin_spee_aco = spotify_sin_string.drop(["Stream", "log_stream", 'Speechiness', 'Acousticness'], axis=1)
y = spotify_sin_string["log_stream"]

In [24]:
baseline_model(X_sin_spee_aco,y,5)

RMSE por fold: [1.62078126 1.60835726 1.59632842 1.58493309 1.60142139]
RMSE medio: 1.6023642852382483


np.float64(1.6023642852382483)

El RMSE medio aumenta de forma más notable respecto al modelo base, lo que
indica que la exclusión de estas variables perjudica el rendimiento del modelo. Por
tanto, se consideran relevantes y se mantienen.

probamos eliminando la variable key

In [25]:
X_sin_key = spotify_sin_string.drop(["Stream", "log_stream","Key"], axis=1)
y = spotify_sin_string["log_stream"]

In [26]:
baseline_model(X_sin_key,y,5)

RMSE por fold: [1.59813894 1.59829515 1.58050255 1.57162887 1.58380021]
RMSE medio: 1.586473143812124


np.float64(1.586473143812124)

A lo largo del proceso iterativo y selección de variables, se han evaluado
distintas combinaciones eliminando tanto variables individuales como pares de variables.
En todos los casos analizados, la eliminación de variables conduce a un empeoramiento o
no mejora del RMSE obtenido mediante validación cruzada.

En consecuencia, **se concluye que todas las variables consideradas aportan información
relevante al modelo** y se decide mantener el conjunto completo de predictores para las
siguientes etapas de modelado. Este resultado sugiere que la mejora del rendimiento
deberá venir de modelos más complejos en lugar de una reducción adicional del conjunto
de variables.

Consideramos que el cambio más significativo en el feature engineering es la transformación logarítmica al target.